## Code that will be used to generate a data frame from which the averages of various performances accross the data sets can be found

In [60]:
import numpy as np
import pandas as pd

# calculate annualised sharpe ratio
def getAnnualisedSR(APY, Rf, numDays):
    return (APY-Rf)/np.sqrt(numDays)
# calculate cumulative return
# this is just the data since we stored this

# calculate APY
def getAPY(data, years):
    """
    Make sure to pass this years that correspond to the current data set
    E.g 254 days per year on average
    """
    # standard here is two years
    Sn = data[-1]
    return np.power(Sn, 1/years) - 1

# calculate MDD
def getMDD(data):
    series = pd.Series(data)
    rollMax = series.cummax()
    dailyDrawDown = series / rollMax - 1
    return dailyDrawDown.cummin().to_numpy().min()

# group to make a single calculation of the metrics
def getMetrics(data, Rf, years, numDays):
    MDD = getMDD(data)
    APY = getAPY(data, years)
    ASR = getAnnualisedSR(APY, Rf, numDays)
    return MDD, APY, ASR

## Code that will be used to retrieve data

In [61]:
def retrieveAllData(exchangeName, startDate, endDate, length):
    # stack as rows on top of each other
    windowSizes = np.arange(start = 10, stop = 200, step = 10)
    allData = np.empty((len(windowSizes), length))
    count = 0
    for i in windowSizes:
        allData[count,:] = np.loadtxt("./{0}/TrainVal/-TRAINVAL-TrainSize-{1}-Start-{2}-End-{3}".format(exchangeName, i, startDate-i, endDate))
        count += 1
    return allData

## Creation of the data frames

In [83]:

# exchanges
exchanges = ["BIS", "BOV", "JSE", "NAS", "SP5"]

# start days
startDays = np.array([200])

# window sizes
windows = np.arange(start = 10, stop = 200, step = 10)
windows = windows.astype(np.int16)

# end days
endDays = np.array([708])

metricsDF = pd.DataFrame(columns = ["TrainSize", "Exchange", "MDD", "APY", "ASR", "Performance"]) 

## Getting all the data

In [84]:
startDate = np.array([200, 1362])
endDate = np.array([708, 1870])
for exchange in exchanges:
    for count in range(2):
        data = retrieveAllData(exchange, startDate[count], endDate[count], 510)
        windowSize = 10
        for i in range(data.shape[0]):
            MDD, APY, ASR = getMetrics(data[i,:], 0.04, 510/252, 510)
            Performance = MDD + APY + ASR
            metricsDF.loc[len(metricsDF.index)] = [windowSize, exchange, MDD, APY, ASR, Performance]
            windowSize += 10
    

In [88]:
metricData = np.empty((19,3))
trainSizeMetrics = pd.DataFrame(columns = ["TrainSize", "MDD", "APY", "ASR", "Performance"]) 
count = 0

for window in windows:
    tempDF = metricsDF[metricsDF["TrainSize"] == window]
    meanMDD = tempDF.MDD.mean()
    meanAPY = tempDF.APY.mean()
    meanASR = tempDF.ASR.mean()
    meanPerformance = tempDF.Performance.mean()
    trainSizeMetrics.loc[len(trainSizeMetrics.index)] = [window, meanMDD, meanAPY, meanASR, meanPerformance]



In [89]:
display(trainSizeMetrics.head(70))
trainSizeMetrics.to_csv('metricsAcrossExchanges.csv')


,TrainSize,MDD,APY,ASR,Performance
0,10.0,-0.277196,0.664773,0.027665,0.415242
1,20.0,-0.255151,0.547023,0.022451,0.314323
2,30.0,-0.289584,0.672485,0.028007,0.410908
3,40.0,-0.249214,0.599939,0.024794,0.375519
4,50.0,-0.273954,0.638963,0.026523,0.391531
5,60.0,-0.312416,0.632239,0.026225,0.346048
6,70.0,-0.287485,0.650193,0.027020,0.389728
7,80.0,-0.282168,0.652032,0.027101,0.396965
8,90.0,-0.303790,0.594312,0.024545,0.315067
9,100.0,-0.326577,0.644046,0.026748,0.344216


In [90]:
jseDF = metricsDF[metricsDF["Exchange"] == "JSE"]
jseDF.head(70)

,TrainSize,Exchange,MDD,APY,ASR,Performance
76,10,JSE,-0.185278,0.353703,0.013891,0.182316
77,20,JSE,-0.269915,0.303468,0.011667,0.045220
78,30,JSE,-0.199876,0.282857,0.010754,0.093735
79,40,JSE,-0.168158,0.337423,0.013170,0.182435
80,50,JSE,-0.187337,0.293717,0.011235,0.117615
81,60,JSE,-0.157698,0.479812,0.019475,0.341589
82,70,JSE,-0.188504,0.377229,0.014933,0.203658
83,80,JSE,-0.188452,0.405942,0.016204,0.233694
84,90,JSE,-0.188504,0.383808,0.015224,0.210528
85,100,JSE,-0.188504,0.440000,0.017712,0.269209
